In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from itertools import combinations
import seaborn as sns

fname='./NanumBarunGothic.otf'
font = fm.FontProperties(fname = fname).get_name()
plt.rcParams["font.family"] = font

board = pd.DataFrame({})
board = pd.read_csv('LT2.csv', encoding='utf-8')
#board = pd.read_csv('LT2_lastweek.csv', encoding='utf-8')
lastRound = len(board.iloc[0,:]) - 1
lastRound

1018

In [6]:

#winnum(r) : r 라운드의 당첨 숫자들 출력
def winnum(r):
    return board.iloc[:, r]

#winnums(end, n): end회 포함 n회동안 등장한 숫자들을 나열한 것(end는 n 이상)
def winnums(end, n):
    ret = board[str(end-n+1)]
    for r in range(end-n+2, end+1):
        ret = ret.append(board[str(r)], ignore_index=True)
    return ret

#func1(end, n) : 1~45 각 숫자가 end회 포함 n회(end - n + 1 ~ end)동안 등장한 횟수 
def func1(end, n):
    ret = winnums(end, n).value_counts()
    #print(ret)
    #print(ret.keys())
    for i in range(1, 46):
        if i not in ret.keys():
            ret[str(i*1.0)] = 0
    return ret

#func2(end, n) : end회 포함 n회동안 0부터 x(0~n)번 사이에 뽑힌 숫자의 개수
def func2(end, n):
    tmp = func1(end, n).value_counts()
    #tmp.loc[0] = 45 - sum(tmp)
    ret = {}
    ret[0] = (tmp[0] if 0 in tmp else 0)
    for i in range(1, n+1):
        ret[i] = ret[i-1] + (tmp[i] if i in tmp else 0)
    return ret
# print(func2(1000,40))
# {0: 0, 1: 0, 2: 4, 3: 7, 4: 15, 5: 25, 6: 32, 7: 39, 8: 43, 9: 45, 10: 45, 11: 45, 12: 45, 13: 45, 14: 45, 15: 45, 16: 45, 17: 45, 18: 45, 19: 45, 20: 45, 21: 45, 22: 45, 23: 45, 24: 45, 25: 45, 26: 45, 27: 45, 28: 45, 29: 45, 30: 45, 31: 45, 32: 45, 33: 45, 34: 45, 35: 45, 36: 45, 37: 45, 38: 45, 39: 45, 40: 45}

#func3(end_start, end_end, n): end_start부터 end_end까지 회를 각각 마지막 회로 하는 연속된 n회에서
#0부터 x(0~n)번까지 뽑힌 숫자의 개수를 각각 func4[x]에 쌓는다.
def func3(end_start, end_end, n):
    end_start = max(end_start, n)
    ret = {}
    for end in range(end_start, end_end + 1):
        func2_result = func2(end, n)
        for i in range(n + 1):
            if i not in ret:
                ret[i] = []
            ret[i].append(func2_result[i])
    for i in range(n + 1):
        ret[i] = np.array(ret[i])
    return pd.Series(ret)
# print(func3(1000, 1010, 6))
# 0    [18, 18, 17, 18, 14, 14, 16, 20, 20, 21, 21]
# 1    [36, 38, 39, 38, 40, 40, 38, 35, 36, 37, 36]
# 2    [45, 43, 43, 43, 45, 45, 45, 44, 43, 41, 42]
# 3    [45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]
# 4    [45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]
# 5    [45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]
# 6    [45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45]

# func4[x][y] : 역사적으로 n번 뽑았을 때 0번부터 x번까지 뽑힌 숫자가 y개 였던 횟수
def func4(n, end_end = lastRound):
    end_start = n
    func3_result = func3(end_start, end_end, n)
    result = [{} for i in range(n+1)]
    for ind in range(len(func3_result)):
        i = func3_result[ind]
        unique, counts = np.unique(i, return_counts=True)
        for i in range(len(unique)):
            result[ind][unique[i]] = counts[i]
    return result

#print(func4(10))
#[{5: 2, 6: 13, 7: 42, 8: 68, 9: 141, 10: 163, 11: 197, 12: 143, 13: 120, 14: 57, 15: 32, 16: 14, 17: 8, 18: 2}, {22: 3, 23: 10, 24: 40, 25: 101, 26: 151, 27: 218, 28: 209, 29: 166, 30: 67, 31: 31, 32: 5, 33: 1}, {34: 6, 35: 16, 36: 65, 37: 155, 38: 211, 39: 248, 40: 170, 41: 97, 42: 26, 43: 6, 44: 2}, {39: 1, 40: 7, 41: 52, 42: 157, 43: 279, 44: 364, 45: 142}, {42: 2, 43: 29, 44: 215, 45: 756}, {43: 2, 44: 33, 45: 967}, {44: 1, 45: 1001}, {45: 1002}, {45: 1002}, {45: 1002}, {45: 1002}]
# func5(n)[x] : n번 뽑았을 때, x번 이하로 나온 개수의 총합의 정상 범위는?
def func5(n, stand):
    func4_result = func4(n)
    ret = [[0, 45] for i in range(n+1)]
    for x in range(n+1):
        # 각 x에 대해 가장 많이 나온 수 구하기
        max_cnt_value = max(func4_result[x].values())
        #max_cnt_value = sum(func4_result[x].values())
        max_cnt_key = -1
        if len(func4_result[x]) == 1:
            ret[x][0] = ret[x][1] = list(func4_result[x].keys())[0]
        for k, v in func4_result[x].items():
            if v == max_cnt_value:
                max_cnt_key = k
                break
                
        # 각 값에 대한 비율로 리스트 치환해, 정상범위 구하기
        
        for i in func4_result[x].keys():
            func4_result[x][i] /= lastRound - n + 1
            if func4_result[x][i] < stand*0.01:
                if i < max_cnt_key:
                    ret[x][0] = i
                else :
                    ret[x][1] = i
                    break
    return ret

#print(func5(10, 5))
#[[5, 17], [23, 32], [34, 43], [40, 45], [43, 45], [44, 45], [44, 45], [45, 45], [45, 45], [45, 45], [45, 45]]
# func6(n) --> n-1번 뽑은 경우에서, n번 뽑았을 때 0~x번 뽑힌 숫자의 개수가 비정상범위인 것들 찾는다.
# 만약, x번 이하로 나온 수가 정상범위 대비 너무 적으면, 다음회차에서, x번 뽑힌 수들은 안나온다.
# x번 이하로 나온 수가 정상범위 대비 너무 크면, 다음회차에서 x번 뽑힌 수중에 정상범위로 들어갈 수 있게 수가 나온다.
def func6(n, stand):
    func5_result = func5(n, stand)
    func2_result = func2(lastRound, n-1)
#     func1_result = winnums(lastRound, n-1)
    func1_result = func1(lastRound, n-1)
#     print(func5_result)
#     print(func2_result)
#     print(func1_result)
#     for k, v in func1_result.items():
#         print(k, v)
    to_picked = [[] for i in range(n+1)]
    to_picked_cnt = [[0, 0] for i in range(n+1)]
    to_removed = []
    for x in range(n):
#         print(x)
        min_limit, max_limit = func5_result[x]
#         print(min_limit, max_limit)
#         print(func2_result[x])
        if func2_result[x] < min_limit:
            for k, v in func1_result.items():
#                 print(k, v)
                if v == x:
                    to_removed.append(k)
                if v < x:
                    break
        elif func2_result[x] > max_limit:
            #print("x : ", x, "max : ", max_limit, "func2[x] : ", func2_result[x])
            to_picked_cnt[x][0] = func2_result[x] - max_limit
            to_picked_cnt[x][1] = func2_result[x] - min_limit
            for k, v in func1_result.items():
                #print(v)
                if v == x:
                    #print("append ", k)
                    to_picked[x].append(k)
                if v < x:
                    break
    return to_picked[:], to_picked_cnt[:], to_removed[:]

# tmp = func6(15, 20)
# print(tmp[0])
# print(tmp[1])
# print(tmp[2])
#[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
# [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
# [5]
#func7(n_from, n_to, stand): func6을 n_from_ n_to 사이의 n들로 돌리면서 to pick, to remove, pick count 누적하기
def func7(n_from, n_to, stand):
    to_picked = []
    to_picked_cnt = []
    to_removed = []
    for n in range(n_from, n_to + 1):
        print(n, len(to_picked), len(to_removed), datetime.datetime.now())
        to_pick, to_pick_cnt, to_remove = func6(n, stand)
#         print(to_pick)
#         print(to_pick_cnt)
#         print(to_remove)
        for i in range(len(to_pick)):
            if len(to_pick[i]) > 0:
                to_picked.append(to_pick[i])
                to_picked_cnt.append(to_pick_cnt[i])
        
        if len(to_remove) > 0:
            for j in to_remove:
                if j not in to_removed:
                    to_removed.append(j)
                    
    return to_picked, to_picked_cnt, to_removed

tmp = func7(7, 37, 8)
print(tmp[0]) 
print(tmp[1])
print(tmp[2])


7 0 0 2022-06-10 23:13:40.653597
8 1 0 2022-06-10 23:13:54.760125
9 1 0 2022-06-10 23:14:11.215229
10 1 0 2022-06-10 23:14:28.446775
11 2 0 2022-06-10 23:14:43.890744
12 2 0 2022-06-10 23:14:58.437992
13 3 0 2022-06-10 23:15:19.934803
14 4 0 2022-06-10 23:15:36.061225
15 5 0 2022-06-10 23:15:49.618078
16 5 0 2022-06-10 23:16:04.658061
17 5 0 2022-06-10 23:16:22.399373
18 5 0 2022-06-10 23:16:37.433895
19 5 0 2022-06-10 23:16:50.651578
20 5 0 2022-06-10 23:17:02.738902
21 5 0 2022-06-10 23:17:14.791939
22 5 0 2022-06-10 23:17:23.544393
23 5 0 2022-06-10 23:17:31.425459
24 5 0 2022-06-10 23:17:38.693861
25 6 0 2022-06-10 23:17:45.749403
26 7 0 2022-06-10 23:17:53.696302
27 8 0 2022-06-10 23:18:00.859071
28 8 0 2022-06-10 23:18:07.998770
29 8 0 2022-06-10 23:18:15.720189
30 8 0 2022-06-10 23:18:23.197383
31 9 0 2022-06-10 23:18:32.118210
32 10 0 2022-06-10 23:18:41.493223
33 11 0 2022-06-10 23:18:49.755332
34 12 4 2022-06-10 23:18:58.128294
35 13 4 2022-06-10 23:19:05.978147
36 14 4 2022-

In [7]:
all_cases = list(combinations([i for i in range(1, 46)], 6))
to_remove = [False for i in range(len(all_cases))]

In [8]:
for ind in range(len(all_cases)):
    i = all_cases[ind]
    if ind%100000 == 0:
        print(ind/len(all_cases), datetime.datetime.now(), to_remove.count(True))
    
    stop_flag = False
    for j in i:
        if j in tmp[2]:
            to_remove[ind] = True
            stop_flag = True
            break
    if stop_flag:
        continue
            
    to_remove_cnt = 0
    for k_ind in range(len(tmp[0])):
        k = tmp[0][k_ind]
        cnt = 0
        for j in i:
            if j in k:
                cnt += 1
        if cnt < tmp[1][k_ind][0]:
            #to_remove[ind] = True
            to_remove_cnt += 1
    if to_remove_cnt / len(tmp[0]) > 0.4:
        to_remove[ind] = True
        
to_remove.count(True)

0.0 2022-06-10 23:19:56.992174 0
0.012277380399898834 2022-06-10 23:19:57.180298 100000
0.024554760799797667 2022-06-10 23:19:58.785259 198093
0.0368321411996965 2022-06-10 23:20:00.665805 295181
0.049109521599595335 2022-06-10 23:20:02.692191 391563
0.06138690199949417 2022-06-10 23:20:03.296887 490864
0.073664282399393 2022-06-10 23:20:05.373295 587576
0.08594166279929184 2022-06-10 23:20:06.690824 685469
0.09821904319919067 2022-06-10 23:20:08.663896 776464
0.11049642359908951 2022-06-10 23:20:10.499432 869176
0.12277380399898834 2022-06-10 23:20:12.218102 963390
0.1350511843988872 2022-06-10 23:20:13.473626 1059668
0.147328564798786 2022-06-10 23:20:13.653071 1159668
0.15960594519868485 2022-06-10 23:20:13.817890 1259668
0.17188332559858369 2022-06-10 23:20:13.984002 1359668
0.18416070599848253 2022-06-10 23:20:14.151152 1459668
0.19643808639838134 2022-06-10 23:20:14.318349 1559668
0.20871546679828018 2022-06-10 23:20:14.483865 1659668
0.22099284719817902 2022-06-10 23:20:14.64305

7808770

In [9]:
remain = []
for i in range(len(all_cases)):
    if not to_remove[i]:
        remain.append(all_cases[i])

In [10]:
lastRound_nums = board.iloc[:, lastRound]
lastRound_nums

0      3.0
1     19.0
2     21.0
3     25.0
4     37.0
5     45.0
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
Name: 1018, dtype: float64

In [11]:
to_remove2 = [False for i in range(len(remain))]
for ind in range(len(remain)):
    i = remain[ind]
    
    even_num = 0
    for j in i:
        if j % 2 == 0:
            even_num += 1
    
    #print("even_num : ", even_num)
    if even_num > 4 or even_num < 2:
        to_remove2[ind] = True
        continue
    
    small_num = 0
    for j in i:
        if j < 23 :
            small_num += 1
            
    #print("small_num : ", small_num)    
    if small_num > 4 or small_num < 2:
        to_remove2[ind] = True
        continue
    
    cont_num = 0
    for j_ind in range(5):
        j = i[j_ind]
        j2 = i[j_ind + 1]
        if j2 - j == 1:
            cont_num += 1
            
    #print("cont_num : ", cont_num)
    if cont_num > 2:
        to_remove2[ind] = True
        continue
        
    last_num = 0
    for j in i:
        if j in lastRound_nums:
            last_num += 1
    
    #print("last_num : ", last_num)
    if last_num > 2:
        to_remove2[ind] = True
        continue
        
to_remove2.count(True)

144331

In [12]:
remain2 = []
for i in range(len(remain)):
    if not to_remove2[i]:
        remain2.append(remain[i])

In [13]:
len(remain2)

191959

In [14]:
num_counts = [0 for i in range(46)]
for i in range(len(remain2)):
    for j in range(0, 46):
        num_counts[j] += remain2[i].count(j)
num_counts

[0,
 21314,
 0,
 21177,
 20468,
 19268,
 0,
 19268,
 21029,
 0,
 20392,
 60663,
 20992,
 24686,
 78881,
 47446,
 28111,
 26217,
 26231,
 0,
 28638,
 26414,
 0,
 32122,
 21665,
 65520,
 74609,
 0,
 19410,
 51957,
 21394,
 0,
 19542,
 17550,
 33850,
 27870,
 0,
 32858,
 0,
 0,
 19501,
 19468,
 42614,
 17014,
 62758,
 60857]

In [15]:
df = pd.DataFrame(remain)
df.to_csv(str(lastRound+1) + ".csv", mode='w')

In [14]:
# 최근 n - 1에 x(0 ~ n - 1)번 등장한 횟수에 각 케이스들을 더해서 n번 동안의 카운트 계산
# 역사적 데이터에서 x(0~n)번 등장한 횟수에 대해 해당 케이스가 차지하는 비율이 어느 정도인지 계산해서
# 가장 정상에 가까운 케이스를 찾는다?
# 하지만, 꼭 너무 가까운 케이스가 좋은 것은 아니진 않을까?
def winnum2(r):
    return board.iloc[:, r]

#winnums(end, n): end회 포함 n회동안 등장한 숫자들을 나열한 것(end는 n 이상)
def winnums2(end, n):
    #print(str(end-n+1))
    ret = board[str(end-n+1)]
    for r in range(end-n+2, end+1):
        ret = ret.append(board[str(r)], ignore_index=True)
    return ret

#func1(end, n) : 1~45 각 숫자가 end회 포함 n회(end - n + 1 ~ end)동안 등장한 횟수 
def func12(end, n):
    return winnums2(end, n).value_counts()

#func2(end, n) : end회 포함 n회동안 0~n번 뽑힌 숫자의 개수
def func22(end, n):
    ret = func12(end, n).value_counts()
    ret.loc[0] = 45 - sum(ret)
    return ret

#func3(end, n) : func2의 결과는 x(0~n)번 뽑힌 숫자의 개수인데, x번 뽑힌 수가 하나도 없다면 func2[x]가 없는 상태이므로, func2[x] = 0 넣어준다. 
def func32(end, n):
    #ret = [0 for i in range(n+1)]
    func2_result = func22(end, n)
    for i in range(n+1):
        if i not in func2_result:
            func2_result[i] = 0
#         else:
#             ret[i] = func2_result.loc[i]
    return func2_result

#func4(end_start, end_end, n): end_start부터 end_end까지 회를 각각 마지막 회로 하는 연속된 n회에서 x(0~n)번 뽑힌 숫자의 개수를 각각 func4[x]에 쌓는다.
#그리고 
def func42(end_start, end_end, n):
    end_start = max(end_start, n)
    ret = {}
    for end in range(end_start, end_end + 1):
        tmp = func32(end, n)
        for i in range(n + 1):
            if i not in ret:
                ret[i] = []
            ret[i].append(tmp[i])
    for i in range(n + 1):
        ret[i] = np.array(ret[i])
    return pd.Series(ret)

# func5[x][y] : 역사적으로 n번 뽑았을 때 x번 뽑힌 숫자가 y개 였던 횟수
def func52(n, end_end = lastRound):
    end_start = n
    func4_result = func42(end_start, end_end, n)
    result = [{} for i in range(n+1)]
    for ind in range(len(func4_result)):
        i = func4_result[ind]
        unique, counts = np.unique(i, return_counts=True)

        for i in range(len(unique)):
            result[ind][unique[i]] = counts[i]

    return result

#func52(5)
# func32(lastRound, 4)
# 1    14
# 2     5
# 0    26
# 3     0
# 4     0
#print(func52(5))
def func8(n, remain):
    func1_result = func12(lastRound, n - 1)
    func5_result = func52(n)
    remain2 = [0 for i in range(len(remain))]
    #print(func1_result[9])
    for i in range(len(remain)):
        if i % 10000 == 0:
            print((i / len(remain)) * 100, datetime.datetime.now())
        tmp = 0
        func1_result2 = func1_result.copy()
        for j in remain[i]:
            if j not in func1_result2:
                func1_result2[j] = 1
            else:
                func1_result2[j] += 1
#         print(func1_result2)
        for j in range(1, 46):
            if j not in func1_result2:
                func1_result2[j] = 0
        func2_result = func1_result2.value_counts()
#         print(func2_result)
#         print(func5_result)
        for j in range(n + 1):
#             print(j)
            if j not in func2_result:
                if 0 in func5_result[j]:
                    tmp += func5_result[j][0]
            else:
                if j in func5_result:
                    if func2_result[j] in func5_result[j]:
                        tmp += func5_result[j][func2_result[j]]
        remain2[i] = tmp
    return remain2

tmp = [0 for i in range(len(remain2))]
for n in range(12, 20):
    print(n)
    tmp2 = func8(n, remain2)
    for i in range(len(tmp)):
        tmp[i] += tmp2[i]

print(tmp)

12
0.0 2022-05-22 11:46:30.127954
0.7322815329581611 2022-05-22 11:48:10.061416
1.4645630659163222 2022-05-22 11:49:50.516182
2.1968445988744834 2022-05-22 11:51:27.288182
2.9291261318326445 2022-05-22 11:52:54.473423
3.6614076647908056 2022-05-22 11:54:18.521991
4.393689197748967 2022-05-22 11:55:28.477880
5.125970730707128 2022-05-22 11:56:38.926441
5.858252263665289 2022-05-22 11:57:48.175224
6.5905337966234505 2022-05-22 11:58:55.767267
7.322815329581611 2022-05-22 12:00:03.345134
8.055096862539772 2022-05-22 12:01:09.583112
8.787378395497933 2022-05-22 12:02:15.889876
9.519659928456093 2022-05-22 12:03:23.268260
10.251941461414257 2022-05-22 12:04:31.078429
10.984222994372416 2022-05-22 12:05:37.759065
11.716504527330578 2022-05-22 12:06:44.442970
12.448786060288738 2022-05-22 12:07:50.250784
13.181067593246901 2022-05-22 12:08:55.954017
13.91334912620506 2022-05-22 12:10:01.757282
14.645630659163222 2022-05-22 12:11:09.169829
15.377912192121382 2022-05-22 12:12:16.836538
16.11019

32.22038745015909 2022-05-22 14:55:10.551189
32.95266898311725 2022-05-22 14:56:01.803070
33.68495051607541 2022-05-22 14:56:53.866837
34.41723204903357 2022-05-22 14:57:45.875517
35.149513581991734 2022-05-22 14:58:37.260664
35.881795114949895 2022-05-22 14:59:28.838157
36.61407664790806 2022-05-22 15:00:20.186720
37.34635818086622 2022-05-22 15:01:12.764636
38.07863971382437 2022-05-22 15:02:07.151254
38.810921246782534 2022-05-22 15:02:59.077056
39.543202779740696 2022-05-22 15:03:51.040707
40.275484312698865 2022-05-22 15:04:42.406087
41.007765845657026 2022-05-22 15:05:36.125324
41.74004737861518 2022-05-22 15:06:29.305935
42.47232891157334 2022-05-22 15:07:20.512072
43.204610444531504 2022-05-22 15:08:12.824345
43.936891977489665 2022-05-22 15:09:03.963728
44.66917351044783 2022-05-22 15:09:55.105387
45.40145504340599 2022-05-22 15:10:48.111542
46.13373657636415 2022-05-22 15:11:40.159705
46.86601810932231 2022-05-22 15:12:31.443638
47.59829964228047 2022-05-22 15:13:22.719022
48

65.17305643327633 2022-05-22 17:33:47.562847
65.9053379662345 2022-05-22 17:35:12.460136
66.63761949919265 2022-05-22 17:36:39.899538
67.36990103215082 2022-05-22 17:38:06.021766
68.10218256510898 2022-05-22 17:39:31.068813
68.83446409806714 2022-05-22 17:40:59.212932
69.56674563102531 2022-05-22 17:42:33.134586
70.29902716398347 2022-05-22 17:43:59.210189
71.03130869694164 2022-05-22 17:45:20.583499
71.76359022989979 2022-05-22 17:46:45.987061
72.49587176285794 2022-05-22 17:48:20.725833
73.22815329581611 2022-05-22 17:49:47.272864
73.96043482877427 2022-05-22 17:51:10.559237
74.69271636173244 2022-05-22 17:52:38.241986
75.42499789469059 2022-05-22 17:54:08.225772
76.15727942764875 2022-05-22 17:55:43.398941
76.88956096060691 2022-05-22 17:57:17.383261
77.62184249356507 2022-05-22 17:58:54.711392
78.35412402652324 2022-05-22 18:00:25.233201
79.08640555948139 2022-05-22 18:01:55.982315
79.81868709243956 2022-05-22 18:03:36.305876
80.55096862539773 2022-05-22 18:05:08.105375
81.28325015

98.12572541639358 2022-05-22 20:59:08.882535
98.85800694935175 2022-05-22 20:59:59.433858
99.5902884823099 2022-05-22 21:00:49.410546
16
0.0 2022-05-22 21:01:36.067454
0.7322815329581611 2022-05-22 21:02:28.548704
1.4645630659163222 2022-05-22 21:03:18.175678
2.1968445988744834 2022-05-22 21:04:06.014711
2.9291261318326445 2022-05-22 21:04:55.219570
3.6614076647908056 2022-05-22 21:05:44.799500
4.393689197748967 2022-05-22 21:06:33.687215
5.125970730707128 2022-05-22 21:07:21.464506
5.858252263665289 2022-05-22 21:08:09.024539
6.5905337966234505 2022-05-22 21:08:56.830911
7.322815329581611 2022-05-22 21:09:43.320592
8.055096862539772 2022-05-22 21:10:30.927314
8.787378395497933 2022-05-22 21:11:17.909415
9.519659928456093 2022-05-22 21:12:04.150613
10.251941461414257 2022-05-22 21:12:50.635195
10.984222994372416 2022-05-22 21:13:37.431650
11.716504527330578 2022-05-22 21:14:23.613670
12.448786060288738 2022-05-22 21:15:10.191519
13.181067593246901 2022-05-22 21:15:57.089072
13.91334912

30.755824384242764 2022-05-22 23:12:15.249685
31.488105917200926 2022-05-22 23:12:44.245314
32.22038745015909 2022-05-22 23:13:13.450561
32.95266898311725 2022-05-22 23:13:42.902013
33.68495051607541 2022-05-22 23:14:11.222248
34.41723204903357 2022-05-22 23:14:39.462361
35.149513581991734 2022-05-22 23:15:07.860040
35.881795114949895 2022-05-22 23:15:36.755473
36.61407664790806 2022-05-22 23:16:05.034809
37.34635818086622 2022-05-22 23:16:33.652519
38.07863971382437 2022-05-22 23:17:01.720120
38.810921246782534 2022-05-22 23:17:30.248715
39.543202779740696 2022-05-22 23:17:58.676161
40.275484312698865 2022-05-22 23:18:27.655532
41.007765845657026 2022-05-22 23:18:55.962712
41.74004737861518 2022-05-22 23:19:24.243484
42.47232891157334 2022-05-22 23:19:52.931612
43.204610444531504 2022-05-22 23:20:21.664125
43.936891977489665 2022-05-22 23:20:50.809209
44.66917351044783 2022-05-22 23:21:19.367133
45.40145504340599 2022-05-22 23:21:48.438641
46.13373657636415 2022-05-22 23:22:17.753541


63.70849336736002 2022-05-23 00:15:23.860244
64.44077490031817 2022-05-23 00:15:34.300090
65.17305643327633 2022-05-23 00:15:44.792322
65.9053379662345 2022-05-23 00:15:55.952038
66.63761949919265 2022-05-23 00:16:06.330032
67.36990103215082 2022-05-23 00:16:16.698780
68.10218256510898 2022-05-23 00:16:27.322959
68.83446409806714 2022-05-23 00:16:37.732113
69.56674563102531 2022-05-23 00:16:48.129798
70.29902716398347 2022-05-23 00:16:58.701373
71.03130869694164 2022-05-23 00:17:09.336419
71.76359022989979 2022-05-23 00:17:20.050491
72.49587176285794 2022-05-23 00:17:30.494494
73.22815329581611 2022-05-23 00:17:41.108358
73.96043482877427 2022-05-23 00:17:51.792137
74.69271636173244 2022-05-23 00:18:02.537132
75.42499789469059 2022-05-23 00:18:13.212618
76.15727942764875 2022-05-23 00:18:23.740105
76.88956096060691 2022-05-23 00:18:34.207980
77.62184249356507 2022-05-23 00:18:45.698645
78.35412402652324 2022-05-23 00:18:56.716116
79.08640555948139 2022-05-23 00:19:07.646482
79.81868709

96.66116235047727 2022-05-23 00:49:59.457713
97.39344388343542 2022-05-23 00:50:10.917623
98.12572541639358 2022-05-23 00:50:22.291942
98.85800694935175 2022-05-23 00:50:34.011324
99.5902884823099 2022-05-23 00:50:44.723422


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
import heapq

remains = []

for i in range(len(tmp)):
    heapq.heappush(remains, (-tmp[i], i))
    
cnt = len(remains)/3
remains_tmp = []
while cnt > 0:
    remains_tmp.append(heapq.heappop(remains))
    cnt -= 1
remains_tmp

[(-80137, 7000),
 (-80137, 7053),
 (-80137, 7176),
 (-80137, 7222),
 (-80137, 7299),
 (-80137, 7334),
 (-80137, 7387),
 (-80137, 7415),
 (-80137, 7486),
 (-80137, 7503),
 (-80137, 7515),
 (-80137, 7522),
 (-80137, 7533),
 (-80137, 7540),
 (-80137, 7542),
 (-80137, 7543),
 (-80137, 7544),
 (-80137, 7546),
 (-80137, 11145),
 (-80137, 11295),
 (-80137, 11372),
 (-80137, 11459),
 (-80137, 11519),
 (-80137, 11588),
 (-80137, 11671),
 (-80137, 11788),
 (-80137, 11817),
 (-80137, 11830),
 (-80137, 11842),
 (-80137, 11853),
 (-80137, 11865),
 (-80137, 11867),
 (-80137, 11868),
 (-80137, 11869),
 (-80137, 11871),
 (-80137, 12273),
 (-80137, 12310),
 (-80137, 12368),
 (-80137, 12396),
 (-80137, 12439),
 (-80137, 12467),
 (-80137, 12528),
 (-80137, 12545),
 (-80137, 12557),
 (-80137, 12564),
 (-80137, 12575),
 (-80137, 12582),
 (-80137, 12584),
 (-80137, 12585),
 (-80137, 12586),
 (-80137, 12588),
 (-80137, 13911),
 (-80137, 13969),
 (-80137, 13997),
 (-80137, 14040),
 (-80137, 14068),
 (-80137, 

In [17]:
last_remain = []
for i in range(int(len(remains_tmp)/2), len(remains_tmp)):
    ind = remains_tmp[i][1]
    last_remain.append(remain2[ind])
df = pd.DataFrame(last_remain)
df.to_csv(str(lastRound+1) + "v2.csv", mode='w')